In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import ipywidgets as widgets
from IPython.display import display, clear_output
from qutip import Bloch

def create_state(theta, phi):
    """Create a normalized qubit state."""
    return np.array([np.cos(theta/2), np.exp(1j * phi) * np.sin(theta/2)])

def measurement_probability(state, basis):
    """Calculate probability of measuring |0⟩ in the given basis."""
    if basis == 'Z':
        return np.abs(state[0])**2
    elif basis == 'X':
        return (1 + np.cos(state[1].imag) * np.sin(state[0].real)) / 2
    elif basis == 'Y':
        return (1 + np.sin(state[1].imag) * np.sin(state[0].real)) / 2

def simulate_measurements(state, basis, n_shots):
    """Simulate measurements in the given basis."""
    prob_0 = measurement_probability(state, basis)
    return np.random.choice([0, 1], size=n_shots, p=[prob_0, 1-prob_0])

def log_likelihood(params, measurements_x, measurements_y, measurements_z):
    """Negative log-likelihood function."""
    theta, phi = params
    state = create_state(theta, phi)
    
    log_l = 0
    for basis, measurements in zip(['X', 'Y', 'Z'], [measurements_x, measurements_y, measurements_z]):
        prob_0 = measurement_probability(state, basis)
        n_0 = np.sum(measurements == 0)
        n_1 = np.sum(measurements == 1)
        log_l += n_0 * np.log(prob_0 + 1e-10) + n_1 * np.log(1 - prob_0 + 1e-10)
    
    return -log_l

def estimate_parameters(measurements_x, measurements_y, measurements_z):
    """Estimate θ and φ using MLE."""
    result = minimize(log_likelihood, [np.pi/2, np.pi], args=(measurements_x, measurements_y, measurements_z),
                      bounds=[(0, np.pi), (0, 2*np.pi)], method='L-BFGS-B')
    return result.x

def fidelity(state1, state2):
    """Compute fidelity between two quantum states."""
    return np.abs(np.vdot(state1, state2))**2

def plot_bloch_sphere(ax, theta, phi, title):
    """Plot Bloch sphere representation on given axes."""
    b = Bloch(axes=ax)
    b.vector_color = ['r']
    b.add_vectors([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])
    b.render()
    ax.set_title(title)

def update_plot(theta, phi, n_shots_x, n_shots_y, n_shots_z):
    """Update interactive visualization."""
    clear_output(wait=True)
    
    true_state = create_state(theta, phi)
    measurements_x = simulate_measurements(true_state, 'X', n_shots_x)
    measurements_y = simulate_measurements(true_state, 'Y', n_shots_y)
    measurements_z = simulate_measurements(true_state, 'Z', n_shots_z)
    
    est_theta, est_phi = estimate_parameters(measurements_x, measurements_y, measurements_z)
    reconstructed_state = create_state(est_theta, est_phi)
    fid = fidelity(true_state, reconstructed_state)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(12, 4))
    
    # True state Bloch sphere
    ax1 = fig.add_subplot(131, projection='3d')
    plot_bloch_sphere(ax1, theta, phi, "True State")
    
    # Reconstructed state Bloch sphere
    ax2 = fig.add_subplot(132, projection='3d')
    plot_bloch_sphere(ax2, est_theta, est_phi, "Reconstructed State")
    
    # Fidelity bar plot
    ax3 = fig.add_subplot(133)
    ax3.bar(['Fidelity'], [fid], color='blue')
    ax3.set_ylim(0, 1)
    ax3.set_title("State Fidelity")
    
    print(f"True θ: {theta:.4f}, φ: {phi:.4f}")
    print(f"Estimated θ: {est_theta:.4f}, φ: {est_phi:.4f}")
    print(f"Fidelity: {fid:.4f}")
    
    plt.tight_layout()
    plt.show()

# Interactive widgets
theta_slider = widgets.FloatSlider(min=0, max=np.pi, step=0.01, value=np.pi/4, description="θ")
phi_slider = widgets.FloatSlider(min=0, max=2*np.pi, step=0.01, value=np.pi/4, description="φ")
n_shots_x_slider = widgets.IntSlider(min=100, max=5000, step=100, value=1000, description="X Shots")
n_shots_y_slider = widgets.IntSlider(min=100, max=5000, step=100, value=1000, description="Y Shots")
n_shots_z_slider = widgets.IntSlider(min=100, max=5000, step=100, value=1000, description="Z Shots")

interactive_plot = widgets.interactive(update_plot, 
                                     theta=theta_slider, 
                                     phi=phi_slider, 
                                     n_shots_x=n_shots_x_slider,
                                     n_shots_y=n_shots_y_slider,
                                     n_shots_z=n_shots_z_slider)

display(interactive_plot)


The above is interactive jupyter code

Below is just normal html save

In [ ]:
import numpy as np
from scipy.optimize import minimize
import plotly.graph_objects as go
def create_state(theta, phi):
    """Create a normalized qubit state
    |ψ⟩ = cos(θ/2)|0⟩ + e^(iφ)sin(θ/2)|1⟩
    """
    return np.array([
        np.cos(theta/2),
        np.exp(1j * phi) * np.sin(theta/2)
    ])

def x_measurement_probability(state):
    """Calculate probability of measuring |+⟩ in the X basis"""
    # X basis projectors for |+⟩
    proj_plus = np.array([[0.5, 0.5], 
                         [0.5, 0.5]])  # Projector for |+⟩
    
    return np.real(state.conj().T @ proj_plus @ state)  # Probability of |+⟩


def y_measurement_probability(state):
    """Calculate probability of measuring |+⟩ in the X basis"""
    # X basis projectors for |+⟩
    proj_plus_y = np.array([[0.5, -1j *0.5], 
                         [0.5j, 0.5]])  # Projector for |+⟩
    
    return np.real(state.conj().T @ proj_plus_y @ state)  # Probability of |+⟩


def simulate_x_measurements(state, n_shots=1000):
    """Simulate measurements in X basis"""
    prob_plus = x_measurement_probability(state)
    return np.random.choice([0, 1], size=n_shots, p=[prob_plus, 1-prob_plus])  # 0 for |+⟩, 1 for |-⟩

def simulate_y_measurements(state, n_shots=1000):
    """Simulate measurements in X basis"""
    prob_plus_y = y_measurement_probability(state)
    return np.random.choice([0, 1], size=n_shots, p=[prob_plus_y, 1-prob_plus_y])  # 0 for |+⟩, 1 for |-⟩


def log_likelihood(params, measurements_x, measurements_z, measurements_y):
    """Calculate log-likelihood for given parameters and X measurements
    The likelihood function for n_plus successes out of n trials is:
    L(θ,φ) = p_plus^(n_plus) * (1-p_plus)^(n_minus)
    where p_plus = probability of measuring |+⟩ in X basis
          = (1 + cos(φ)*sin(θ))/2
    """
    
    theta, phi = params
    state = create_state(theta, phi)
    
    # Probability of measuring |+⟩ in X basis
    prob_plus_x_0 = ((1 + np.cos(phi) * np.sin(theta)) / 2 )
    prob_plus_Z_0= z_measurement_probability(state)
    prob_plus_y_0= (1+ np.sin(theta)*np.sin(phi))/2
    n_0_z = np.sum(measurements_z == 0)  # |0⟩ outcome count
    n_1_z = np.sum(measurements_z == 1)  # |1⟩ outcome count
    
    # Count number of |+⟩ and |-⟩ measurements
    n_plus_x = len(measurements_x) - np.sum(measurements_x)  # |+⟩ outcome count
    n_minus_x = np.sum(measurements_x)  # |-⟩ outcome count
    
    n_0_y = np.sum(measurements_y == 0)  # |0⟩ outcome count
    n_1_y = np.sum(measurements_y == 1)  # |1⟩ outcome count
    
    log_1_z = n_0_z * np.log(prob_plus_Z_0 + 1e-10) + n_1_z * np.log(1 - prob_plus_Z_0 + 1e-10)
 
    log_1_x = n_plus_x * np.log(prob_plus_x_0 + 1e-10) + n_minus_x * np.log(1 - prob_plus_x_0 + 1e-10)
    log_1_y=n_0_y*np.log(prob_plus_y_0 + 1e-10) + n_1_y * np.log(1 - prob_plus_y_0 + 1e-10)
    log_1= log_1_z+ log_1_x+ log_1_y
    
    return -log_1  # Return negative for minimization

def theoretical_x_probability(theta, phi):
    """Calculate theoretical probability of |+⟩ in X basis
    p_plus = (1 + cos(φ)*sin(θ))/2
    """
    return (1 + np.cos(phi) * np.sin(theta)) / 2

def main():
    # Create a true state
    true_theta = 2.14 # 30 degrees
    true_phi = 3.96    # 20 degrees (relevant in X basis)
    true_state = create_state(true_theta, true_phi)
    
    # Simulate X measurements
    n_shots = 5000
    measurements_x = simulate_x_measurements(true_state,n_shots=n_shots)
    measurements_z = simulate_z_measurements(true_state, n_shots=n_shots)
    measurements_y = simulate_y_measurements(true_state, n_shots=n_shots)
    #print(measurements)
    #print("length of measurements:")
    #print(len(measurements))
    
    # Calculate observed frequency of |+⟩
    #observed_freq = (len(measurements) - np.sum(measurements)) / len(measurements)
    
    # Estimate parameters using MLE
    initial_params = [np.pi/3, np.pi/2]  # Initial guess
    bounds = [(1e-6, np.pi), (1e-6, 2*np.pi)]  # Bounds for (θ, φ)
    
    result = minimize(log_likelihood, initial_params, 
                  args=(measurements_x,measurements_z, measurements_y),
                  bounds=bounds,
                  method='L-BFGS-B',
                  options={'ftol': 1e-6, 'gtol': 1e-6})
    
    est_theta, est_phi = result.x  # Estimated parameters
    
    # Print results
    print(f"True parameters: θ = {true_theta:.8f}, φ = {true_phi:.8f}")
    print(f"True X-basis |+⟩ probability: {theoretical_x_probability(true_theta, true_phi):.8f}")
    #print(f"\nObserved frequency of |+⟩: {observed_freq:.8f}")
    print(f"\nEstimated parameters: θ = {est_theta:.8f}, φ = {est_phi:.8f}")
    print(f"Estimated X-basis |+⟩ probability: {theoretical_x_probability(est_theta, est_phi):.8f}")
    print(f"Estimated X-basis |+⟩ probability: {theoretical_x_probability(est_theta, est_phi):.8f}")

# Run the main function
if __name__ == "__main__":
    main()

In [ ]:


def main():
    # Define a range of true theta and phi values
    theta_values = np.linspace(0.01, np.pi-0.01, 20)
    phi_values = np.linspace(0.01, 2*np.pi-0.01, 20)

    fidelities = np.zeros((len(theta_values), len(phi_values)))

    for i, true_theta in enumerate(theta_values):
        for j, true_phi in enumerate(phi_values):
            # Create a true state
            true_state = create_state(true_theta, true_phi)
            n_shots = 5000
            measurements_x = simulate_x_measurements(true_state, n_shots=n_shots)
            measurements_z = simulate_z_measurements(true_state, n_shots=n_shots)
            measurements_y = simulate_y_measurements(true_state, n_shots=n_shots)

            bounds = [(1e-6, np.pi), (1e-6, 2*np.pi)]  # Bounds for (θ, φ)
            best_fide = 0

            for _ in range(20):  # Try 20 random initial parameters
                # Generate random initial parameters within the bounds
                initial_params = np.random.uniform(low=[b[0] for b in bounds], high=[b[1] for b in bounds])

                result = minimize(log_likelihood, initial_params, 
                                  args=(measurements_x, measurements_z, measurements_y),
                                  bounds=bounds,
                                  options={'ftol': 1e-6, 'gtol': 1e-6})

                est_theta, est_phi = result.x  # Estimated parameters
                est_state = create_state(est_theta, est_phi)
                fide = abs(est_state.conj().T @ true_state)**2

                if fide > best_fide:
                    best_fide = fide

            fidelities[i, j] = best_fide

    print(fidelities)


    theta_grid, phi_grid = np.meshgrid(theta_values, phi_values)

    fig = go.Figure(data=[go.Surface(z=fidelities.T, x=theta_grid, y=phi_grid)])
    fig.update_layout(
    title='multiple projections including Pauli Y for 5000 shots with 20 random initial params ',
    scene=dict(
        xaxis_title='True Theta',
        yaxis_title='True Phi',
        zaxis_title='Fidelity',
        xaxis=dict(range=[min(theta_values), max(theta_values)]),
        yaxis=dict(range=[min(phi_values), max(phi_values)]),
        zaxis=dict(range=[0, 1]),  # Set z-axis range from 0 to 1
    ),width=800,height=800
    )


    # Save the figure to an HTML file
    fig.write_html("fidelity_plot_new11.html")

if __name__ == "__main__":
    main()
